<a href="https://colab.research.google.com/github/HedersonSantos/nlp/blob/master/LLM/fine_tune_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflow as tf
import matplotlib.pyplot as plt

# Carregue o modelo pré-treinado
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Carregue seus dados de texto
text_data = open('seu_arquivo.txt').read()

# Tokenize o texto
input_ids = tokenizer.encode(text_data, return_tensors='tf')

# Prepare as sequências de entrada e saída
input_sequences = input_ids[:, :-1]
output_sequences = input_ids[:, 1:]

# Compile o modelo
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

# Divida os dados em conjuntos de treinamento e validação
train_size = int(0.8 * len(input_sequences))
train_input_sequences = input_sequences[:train_size]
train_output_sequences = output_sequences[:train_size]
val_input_sequences = input_sequences[train_size:]
val_output_sequences = output_sequences[train_size:]

# Treine o modelo
epochs = 3
batch_size = 16
history = model.fit(train_input_sequences, train_output_sequences, epochs=epochs, batch_size=batch_size, validation_data=(val_input_sequences, val_output_sequences))

# Avalie o modelo
loss, accuracy = model.evaluate(val_input_sequences, val_output_sequences, batch_size=batch_size)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')

# Plote o gráfico com a descida do gradiente com o valor da função loss para o treinamento e validação
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Salve o modelo
model.save_pretrained('seu_modelo')
